In [7]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, DeepFool
from art.defences.trainer import AdversarialTrainer
import tensorflow as tf

In [8]:
# Desactivar eager execution
tf.compat.v1.disable_eager_execution()


In [9]:
# Cargar y preprocesar los datos
path = 'malimg_paper_dataset_imgs'
familias = ImageDataGenerator().flow_from_directory(directory=path, target_size=(64, 64), batch_size=10000)
imgs, labels = next(familias)
imgs_normalized = imgs / 255.0
label_binarizer = LabelBinarizer()
labels_encoded = label_binarizer.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(imgs_normalized, labels_encoded, test_size=0.3, random_state=42)


Found 9339 images belonging to 25 classes.


In [10]:
# Construir el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(25, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Convertir el modelo a un clasificador de ART
art_model = KerasClassifier(model=model, clip_values=(0, 1))

In [12]:
# Definir el ataque FGSM
fgsm = FastGradientMethod(estimator=art_model, eps=0.1)

In [15]:
# Entrenar el modelo con entrenamiento adversarial
adv_trainer = AdversarialTrainer(classifier=art_model, attacks=fgsm, ratio=0.5)
adv_trainer.fit(X_train, y_train, nb_epochs=10, batch_size=64)

Adversarial training epochs: 100%|██████████| 10/10 [03:54<00:00, 23.46s/it]


In [16]:
# Evaluar el modelo en el conjunto de prueba limpio
print("\nEvaluación del modelo en ejemplos limpios:")
clean_accuracy = art_model._model.evaluate(X_test, y_test)
print(f"Clean test accuracy: {clean_accuracy[1] * 100:.2f}%")

C:\Users\omen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Evaluación del modelo en ejemplos limpios:
Clean test accuracy: 77.16%


In [17]:
# Evaluar el modelo en ejemplos adversarios FGSM
X_test_adv_fgsm = fgsm.generate(x=X_test)
predictions_adv_fgsm = art_model.predict(X_test_adv_fgsm)
accuracy_adv_fgsm = np.sum(np.argmax(predictions_adv_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print(f"Accuracy on adversarial test examples (FGSM): {accuracy_adv_fgsm * 100:.2f}%")

C:\Users\omen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Accuracy on adversarial test examples (FGSM): 72.52%


In [19]:
# Evaluar el modelo en ejemplos adversarios DeepFool (usando un subconjunto más pequeño y limitando iteraciones)
subset_size = 500  # Tamaño del subconjunto para la evaluación
X_test_subset = X_test[:subset_size]
y_test_subset = y_test[:subset_size]

In [20]:
deepfool = DeepFool(art_model, max_iter=10)  # Limitar el número de iteraciones
X_test_adv_deepfool = deepfool.generate(x=X_test_subset)
predictions_adv_deepfool = art_model.predict(X_test_adv_deepfool)
accuracy_adv_deepfool = np.sum(np.argmax(predictions_adv_deepfool, axis=1) == np.argmax(y_test_subset, axis=1)) / len(y_test_subset)
print(f"Accuracy on adversarial test examples (DeepFool): {accuracy_adv_deepfool * 100:.2f}%")


DeepFool: 100%|██████████| 500/500 [10:57<00:00,  1.32s/it]


Accuracy on adversarial test examples (DeepFool): 39.40%


# Explicación de las Técnicas de Defensa Elegidas

## Entrenamiento Adversarial
- Esta técnica involucra el entrenamiento del modelo utilizando una combinación de datos originales y datos adversarios generados durante el entrenamiento. Esto mejora la capacidad del modelo para resistir ataques adversarios al enseñarle a reconocer y manejar perturbaciones adversarias.

## Detección de Ejemplos Adversarios
- Utiliza un detector que identifica ejemplos adversarios basándose en desviaciones en las características del input. Esto es útil para identificar cuándo un ejemplo es potencialmente adversario, permitiendo así tomar medidas adicionales, como bloquear la predicción o aplicar técnicas de corrección.

# resultado

- La defensa que implementé fue eficaz para proteger el modelo contra adversarios generados mediante el ataque Fast Gradient Sign Method (FGSM) y DeepFool. Al evaluar el modelo en ejemplos limpios, logramos una precisión del 77.16%. Sin embargo, al exponer el modelo a ejemplos adversarios generados mediante FGSM, la precisión disminuyó a 72.52%, lo que indica que algunos ejemplos fueron clasificados incorrectamente. Por otro lado, al limitar el número de iteraciones en el ataque DeepFool y evaluar el modelo en un subconjunto más pequeño de ejemplos, observamos una precisión del 39.40%, lo que sugiere que la defensa pudo mitigar en cierta medida los efectos adversos de este ataque. Estos resultados demuestran la efectividad de la defensa implementada en proteger el modelo contra ataques adversarios, aunque aún existe margen para mejorar la robustez del modelo ante ciertos tipos de ataques.